In [29]:
import os
import pandas as pd

df = pd.read_csv("zoom_logs/8기 파문기_1주차.csv")

In [30]:
df

,이름(원래 이름),사용자 이메일,참가 시간,나간 시간,기간(분),게스트,대기실
0,jun datachef,official.datachef@gmail.com,2024/02/28 08:56:29 PM,2024/02/28 09:55:51 PM,60,아니요,아니요
1,박원미_51 (박원미 건국대학교),NaN,2024/02/28 08:56:57 PM,2024/02/28 09:55:51 PM,59,예,아니요
2,손승은 51 (승은 손),NaN,2024/02/28 08:56:57 PM,2024/02/28 09:55:50 PM,59,예,아니요
3,안재경_77 (안재경),NaN,2024/02/28 08:56:59 PM,2024/02/28 09:55:51 PM,59,예,아니요
4,김세영_14 (202112324 김세영 스마트ict융합공학과),NaN,2024/02/28 08:57:00 PM,2024/02/28 09:10:07 PM,14,예,아니요
...,...,...,...,...,...,...,...
224,이찬희_47,NaN,2024/02/28 09:38:17 PM,2024/02/28 09:40:30 PM,3,예,아니요
225,이찬희_47,NaN,2024/02/28 09:40:34 PM,2024/02/28 09:42:07 PM,2,예,아니요
226,강빈_21 (KANG BIN),NaN,2024/02/28 09:44:21 PM,2024/02/28 09:55:54 PM,12,예,아니요
227,심재현_93,NaN,2024/02/28 08:59:34 PM,2024/02/28 09:55:54 PM,57,예,아니요


In [50]:
df = pd.read_csv("zoom_logs/8기 파문기_1주차.csv")
jun_time = df[df['게스트'] == '아니요']['기간(분)'][0]

In [31]:
df = pd.read_csv("zoom_logs/8기 파문기_1주차.csv")
jun_time = df[df['게스트'] == '아니요']['기간(분)'][0]
df['이름(원래 이름)'] = df['이름(원래 이름)'].apply(lambda x : x.split('(')[0].replace(' ','').replace('_',''))

In [32]:
df.groupby(['이름(원래 이름)'])['기간(분)'].sum().reset_index().head(20)

,이름(원래 이름),기간(분)
0,.,25
1,8기김예린12,55
2,8기이원준09,48
3,8기황정현60,6
4,JiYoungLee,1
5,jundatachef,60
6,강민규76,57
7,강민채13,56
8,강빈21,12
9,강수빈08,58


In [33]:
import gspread
import os
from dotenv import load_dotenv

# json 파일이 위치한 경로를 값으로 줘야 합니다.

json_file_path = os.environ.get('GCP_JSON_PATH')

gc = gspread.service_account(json_file_path)

spreadsheet_url = os.environ.get('SHEET_URL')
doc = gc.open_by_url(spreadsheet_url)

main_worksheet = doc.worksheet("총명단")
python_attendance = doc.worksheet("Python 문법 기초반")

In [42]:
df = pd.read_csv("zoom_logs/8기 파문기_1주차.csv")
jun_time = df[df['게스트'] == '아니요']['기간(분)'][0]
df['이름(원래 이름)'] = df['이름(원래 이름)'].apply(lambda x : x.split('(')[0].replace(' ','').replace('_',''))
range_list = [x.value for x in python_attendance.range('A4:A400')]
merge_df = pd.merge(pd.DataFrame(range_list, columns=['이름(원래 이름)']), df.groupby(['이름(원래 이름)'])['기간(분)'].sum().reset_index(), on='이름(원래 이름)', how='left').fillna(0)
merge_df

,이름(원래 이름),기간(분)
0,강빈21,12.0
1,강서윤69,0.0
2,강수빈08,58.0
3,강예은46,59.0
4,강지웅11,57.0
...,...,...
292,,0.0
293,,0.0
294,,0.0
295,,0.0


In [53]:
merge_df['출첵'] = merge_df['기간(분)'].apply(lambda x : 'O' if jun_time-10 <= x else ('지각' if x != 0 else 'X'))
merge_df[merge_df['이름(원래 이름)'] != '']

,이름(원래 이름),기간(분),출첵
0,강빈21,12.0,지각
1,강서윤69,0.0,X
2,강수빈08,58.0,O
3,강예은46,59.0,O
4,강지웅11,57.0,O
...,...,...,...
220,한소리67,56.0,O
221,한혜인88,0.0,X
222,황정현60,54.0,O
223,황지희73,0.0,X
